In [112]:
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import math

import itertools

In [113]:
import urllib.request
import zipfile
import pandas as pd
from io import StringIO
import os
import logging
import pathlib

logging.basicConfig(level=logging.INFO)


def get_cache_data(dataset, local_cache_path):
    local_cache_path = pathlib.Path(local_cache_path).expanduser()
    if read_pickle_or_none(local_cache_path) is None:
        df = get_data(dataset)
        logging.info(f"Writing data to local cache file {local_cache_path}")
        df.to_pickle(local_cache_path)
    else:
        logging.info(f"Reading local cache file {local_cache_path}")
        df = pd.read_pickle(local_cache_path)
    return df


def get_data(dataset):
    """
    download, concat and return df of specified dataset
    parameters
    dataset: ["Inpatient", "Outpatient]
    """
    base_url = f"https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/SynPUFs/Downloads/DE1_0_2008_to_2010_{dataset}_Claims_Sample_"
    df = pd.DataFrame()
    for i in range(1, 21):
        logging.info(f"Fetching file {dataset} {i}...")
        url = f"{base_url}{i}.zip"
        file, _ = urllib.request.urlretrieve(url)
        _df = read_from_zip(file)
        df = pd.concat([df, _df])
    # convert date fields to datetime type
    for c in [c for c in df.columns if "_DT" in c]:
        df[c] = pd.to_datetime(df[c])
    # convert amount fields to float type
    for c in [c for c in df.columns if "_AMT" in c]:
        df[c] = df[c].astype(float)
    return df


def read_from_zip(file):
    zip = zipfile.ZipFile(file, "r")
    first_file = zip.namelist()[0]
    file = zip.open(first_file)
    content = file.read()
    s = str(content, "utf-8")
    data = StringIO(s)
    return pd.read_csv(data, dtype=str)


def read_pickle_or_none(FILE) -> pd.DataFrame:
    if os.path.isfile(FILE):
        result = pd.read_pickle(FILE)
    else:
        result = None
    return result

In [123]:
outpatient = get_cache_data("Outpatient", "Outpatient.pkl")


INFO:root:Reading local cache file Outpatient.pkl


In [ ]:
outpatient.dropna(subset=['CLM_PMT_AMT'], inplace=True)
outpatient.dropna(subset=['CLM_FROM_DT', 'CLM_THRU_DT'], inplace=True)
outpatient.dropna(subset=['ICD9_DGNS_CD_1', 'ICD9_DGNS_CD_2', 
'ICD9_DGNS_CD_3', 'ICD9_DGNS_CD_4', 'ICD9_DGNS_CD_5', 'ICD9_DGNS_CD_6', 'ICD9_DGNS_CD_7',
 'ICD9_DGNS_CD_8', 'ICD9_DGNS_CD_9','ICD9_DGNS_CD_10'], inplace=True, how='all')
outpatient.dropna(subset=['ICD9_PRCDR_CD_1', 'ICD9_PRCDR_CD_2', 
 'ICD9_PRCDR_CD_3', 'ICD9_PRCDR_CD_4', 'ICD9_PRCDR_CD_5', 'ICD9_PRCDR_CD_6'], inplace=True, how='all')
outpatient.dropna(subset=['HCPCS_CD_1', 
 'HCPCS_CD_2', 'HCPCS_CD_3', 'HCPCS_CD_4', 'HCPCS_CD_5', 'HCPCS_CD_6', 
 'HCPCS_CD_7', 'HCPCS_CD_8', 'HCPCS_CD_9',
 'HCPCS_CD_10', 'HCPCS_CD_11', 'HCPCS_CD_12', 'HCPCS_CD_13', 'HCPCS_CD_14',
 'HCPCS_CD_15', 'HCPCS_CD_16', 'HCPCS_CD_17', 'HCPCS_CD_18', 'HCPCS_CD_19',
 'HCPCS_CD_20', 'HCPCS_CD_21', 'HCPCS_CD_22', 'HCPCS_CD_23', 'HCPCS_CD_24',
 'HCPCS_CD_25', 'HCPCS_CD_26', 'HCPCS_CD_27', 'HCPCS_CD_28', 'HCPCS_CD_29',
 'HCPCS_CD_30', 'HCPCS_CD_31', 'HCPCS_CD_32', 'HCPCS_CD_33', 'HCPCS_CD_34',
 'HCPCS_CD_35', 'HCPCS_CD_36', 'HCPCS_CD_37', 'HCPCS_CD_38', 'HCPCS_CD_39',
 'HCPCS_CD_40', 'HCPCS_CD_41', 'HCPCS_CD_42', 'HCPCS_CD_43', 'HCPCS_CD_44',
 'HCPCS_CD_45'], inplace=True, how='all')

In [124]:
column_sets_dict = {'clm_dates': ['CLM_FROM_DT', 'CLM_THRU_DT'],
'provider': ['PRVDR_NUM'],
'DGNS_CD': ['ICD9_DGNS_CD_1', 'ICD9_DGNS_CD_2', 'ICD9_DGNS_CD_3',
 'ICD9_DGNS_CD_4', 'ICD9_DGNS_CD_5', 'ICD9_DGNS_CD_6', 'ICD9_DGNS_CD_7',
 'ICD9_DGNS_CD_8', 'ICD9_DGNS_CD_9','ICD9_DGNS_CD_10'],
'PRDCR_CD': ['ICD9_PRCDR_CD_1', 'ICD9_PRCDR_CD_2', 'ICD9_PRCDR_CD_3',
 'ICD9_PRCDR_CD_4', 'ICD9_PRCDR_CD_5', 'ICD9_PRCDR_CD_6'],
'HCPCS_CD': ['HCPCS_CD_1', 'HCPCS_CD_2', 'HCPCS_CD_3', 'HCPCS_CD_4',
 'HCPCS_CD_5', 'HCPCS_CD_6', 'HCPCS_CD_7', 'HCPCS_CD_8', 'HCPCS_CD_9',
 'HCPCS_CD_10', 'HCPCS_CD_11', 'HCPCS_CD_12', 'HCPCS_CD_13', 'HCPCS_CD_14',
 'HCPCS_CD_15', 'HCPCS_CD_16', 'HCPCS_CD_17', 'HCPCS_CD_18', 'HCPCS_CD_19',
 'HCPCS_CD_20', 'HCPCS_CD_21', 'HCPCS_CD_22', 'HCPCS_CD_23', 'HCPCS_CD_24',
 'HCPCS_CD_25', 'HCPCS_CD_26', 'HCPCS_CD_27', 'HCPCS_CD_28', 'HCPCS_CD_29',
 'HCPCS_CD_30', 'HCPCS_CD_31', 'HCPCS_CD_32', 'HCPCS_CD_33', 'HCPCS_CD_34',
 'HCPCS_CD_35', 'HCPCS_CD_36', 'HCPCS_CD_37', 'HCPCS_CD_38', 'HCPCS_CD_39',
 'HCPCS_CD_40', 'HCPCS_CD_41', 'HCPCS_CD_42', 'HCPCS_CD_43', 'HCPCS_CD_44',
 'HCPCS_CD_45']}

In [125]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
percent_missing = outpatient.isnull().sum() * 100 / len(outpatient)
missing_value_df = pd.DataFrame({'column_name': outpatient.columns,
                                 'percent_missing': percent_missing})
display(outpatient.head(100))
display(missing_value_df.head(83))

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,AT_PHYSN_NPI,OP_PHYSN_NPI,OT_PHYSN_NPI,NCH_BENE_BLOOD_DDCTBL_LBLTY_AM,ICD9_DGNS_CD_1,ICD9_DGNS_CD_2,ICD9_DGNS_CD_3,ICD9_DGNS_CD_4,ICD9_DGNS_CD_5,ICD9_DGNS_CD_6,ICD9_DGNS_CD_7,ICD9_DGNS_CD_8,ICD9_DGNS_CD_9,ICD9_DGNS_CD_10,ICD9_PRCDR_CD_1,ICD9_PRCDR_CD_2,ICD9_PRCDR_CD_3,ICD9_PRCDR_CD_4,ICD9_PRCDR_CD_5,ICD9_PRCDR_CD_6,NCH_BENE_PTB_DDCTBL_AMT,NCH_BENE_PTB_COINSRNC_AMT,ADMTNG_ICD9_DGNS_CD,HCPCS_CD_1,HCPCS_CD_2,HCPCS_CD_3,HCPCS_CD_4,HCPCS_CD_5,HCPCS_CD_6,HCPCS_CD_7,HCPCS_CD_8,HCPCS_CD_9,HCPCS_CD_10,HCPCS_CD_11,HCPCS_CD_12,HCPCS_CD_13,HCPCS_CD_14,HCPCS_CD_15,HCPCS_CD_16,HCPCS_CD_17,HCPCS_CD_18,HCPCS_CD_19,HCPCS_CD_20,HCPCS_CD_21,HCPCS_CD_22,HCPCS_CD_23,HCPCS_CD_24,HCPCS_CD_25,HCPCS_CD_26,HCPCS_CD_27,HCPCS_CD_28,HCPCS_CD_29,HCPCS_CD_30,HCPCS_CD_31,HCPCS_CD_32,HCPCS_CD_33,HCPCS_CD_34,HCPCS_CD_35,HCPCS_CD_36,HCPCS_CD_37,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_CD_45
1630,0071E2E641B73233,542272281121304,1,2008-03-12,2008-03-16,2600ZT,200.0,0.0,NaN,NaN,NaN,0.00,4848,78053,5859,4110,5856,NaN,NaN,NaN,NaN,NaN,3995,NaN,NaN,NaN,NaN,NaN,0.0,10.0,NaN,A9270,84166,84155,84165,NaN,86334,86334,85025,84166,86235,86618,82784,85652,84155,83690,84155,84165,86334,83001,84156,82785,85027,84165,86235,P9045,82785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1688,0073EAD53F4BBC1C,542442281542736,1,2010-08-21,2010-08-24,4313WN,500.0,0.0,6547194871,6547194871,6547194871,0.00,42731,V462,84200,2819,V442,41401,78079,NaN,NaN,NaN,7971,V5861,2720,NaN,NaN,NaN,0.0,1100.0,NaN,36415,85018,85014,86850,J0885,83883,85018,85610,85025,87086,85610,82550,84443,84484,A9270,83735,36415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1730,0076A3B03FA644E9,542222280894691,1,2010-04-07,2010-04-27,1000YC,600.0,0.0,2490248597,3197072629,NaN,0.00,51882,V4501,496,4019,42731,V5861,78039,03819,5990,NaN,3324,NaN,NaN,NaN,NaN,NaN,0.0,200.0,NaN,94760,85027,83970,80053,84520,80051,85610,85610,99212,80048,NaN,84436,84295,93005,80048,80053,NaN,NaN,93005,80048,85045,80164,83735,80061,85576,85027,85025,82550,81003,80053,83735,81003,85610,NaN,80048,85025,81001,NaN,NaN,85025,81001,83550,NaN,80053,NaN
3385,00FDFA655DD462F8,542972280966533,1,2008-02-06,2008-02-06,14026N,20.0,0.0,3808196039,8126918434,5579681385,0.00,V045,V700,78900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9952,V1043,NaN,NaN,NaN,NaN,0.0,0.0,NaN,90658,G0008,90945,G0008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4255,013C8094EEB51A84,542402280840167,1,2008-01-18,2008-01-19,1600GH,60.0,0.0,0290865274,0290865274,NaN,0.00,29663,V5869,4019,3019,30183,NaN,NaN,NaN,NaN,NaN,9462,NaN,NaN,NaN,NaN,NaN,0.0,80.0,NaN,90853,NaN,80053,80061,84153,82607,77523,85610,87088,85014,85610,83036,85025,86922,82728,80076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20008,0620CE3E8E167617,542592281534122,1,2008-10-22,2008-10-23,2100YG,20.0,0.0,0581375878,9421678738,NaN,0.00,5579,40391,V140,V4582,3051,40390,4019,2449,45989,NaN,3929,NaN,NaN,NaN,NaN,NaN,0.0,1100.0,NaN,36415,36200,NaN,80048,82947,85730,84439,Q9962,82947,J2250,J2405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23095,072A6918C8486FAC,542242281535363,1,2009-06-16,2009-06-17,5000WD,3300.0,0.0,9481160337,2459829986,NaN,0.00,99602,7869,41400,4019,78057,NaN,NaN,NaN,NaN,NaN,0066,7820,412,78650,71595,7840,0.0,1100.0,NaN,NaN,36415,81003,85025,96372,36415,82962,85025,96366,84484,96375,99282,77413,36415,99283,93306,86900,99285,85025,83874,94260,J0690,94640,85610,87086,A9541,96374,93005,96375,93510,99283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42778,0D3B52D303876737,542702281277652,1,2008-06-2

,column_name,percent_missing
DESYNPUF_ID,DESYNPUF_ID,0.000000
CLM_ID,CLM_ID,0.000000
SEGMENT,SEGMENT,0.000000
CLM_FROM_DT,CLM_FROM_DT,0.000000
CLM_THRU_DT,CLM_THRU_DT,0.000000
PRVDR_NUM,PRVDR_NUM,0.000000
CLM_PMT_AMT,CLM_PMT_AMT,0.000000
NCH_PRMRY_PYR_CLM_PD_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,0.000000
AT_PHYSN_NPI,AT_PHYSN_NPI,1.059374
OP_PHYSN_NPI,OP_PHYSN_NPI,3.350579


In [137]:
def isnan(x):
    return x != x

def TFIDF_Matrix(df, columns):
    #Takes in a dataframe, and a list of columns that comprise a "sentence"
    #Returns a TFIDF matrix

    corpus = []

    if len(columns) > 1:
        values = df[columns].values.tolist()
        for sentence in values:
            s = ''
            for word in sentence:
                if len(s) == 0:
                    if isnan(word):
                        s = 'None'
                    else:
                        s = str(word)
                else:
                    if isnan(word):
                        s += 'None'
                    else:
                        s += str(word)
            corpus.append(s)

    else:
        values = df[columns].values.tolist()
        for word in values:
            if isnan(word[0]):
                corpus.append('None')
            else:
                corpus.append(word[0])
    
    return  TfidfVectorizer(stop_words=['None']).fit_transform(corpus).todense()

def Date_Diff(df, columns):
    #Takes in a dataframe, and a list of columns that are a start date and end date
    #Returns the difference between the two dates

    values = df[columns]

    values['date_diff'] = (df[columns[1]] - df[columns[0]]) / np.timedelta64(1,'D')

    return  np.array(values['date_diff'].values.tolist()).reshape(len(values),1)
    
def Passthrough(df, columns):
    #Takes in a dataframe, and a list of columns that need to be turned into a list
    #Returns list of values from columns
    
    return np.array(df[columns].values.tolist())

In [138]:
column_sets_matrix_dict = {}
for key, value in column_sets_dict.items():
    print(key)
    if key in ['provider', 'NPI', 'admit_code', 'claim_discharge_code', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD']:
        column_sets_matrix_dict[key] = TFIDF_Matrix(outpatient, value)
    elif key in ['clm_dates', 'admit_dates']:
        column_sets_matrix_dict[key] = Date_Diff(outpatient, value)
    else:
        column_sets_matrix_dict[key] = Passthrough(outpatient, value)

clm_dates
provider
DGNS_CD
PRDCR_CD
HCPCS_CD


In [144]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

def example_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    GDBreg = GradientBoostingRegressor(n_estimators = 100, learning_rate=0.1, max_depth = 3, random_state = 0, loss = 'ls').fit(X_train, y_train)
    return mean_squared_error(y_test, GDBreg.predict(X_test))

In [145]:
results = {}
for n_met in range(len(column_sets_matrix_dict)):
    for met_set in list(itertools.combinations(column_sets_matrix_dict.keys(), n_met+1)):
        for n_met_set in range(len(met_set)):
            if n_met_set == 0:
                X = column_sets_matrix_dict[met_set[n_met_set]]
            else:
                X  = np.hstack((X,column_sets_matrix_dict[met_set[n_met_set]]))
        answer = example_model(X, outpatient['CLM_PMT_AMT'].values.tolist())
        results[met_set] = answer

In [146]:
print(results)

{('clm_dates',): 663631.3482178181, ('provider',): 679591.9600154649, ('DGNS_CD',): 660811.000556215, ('PRDCR_CD',): 579708.3787235251, ('HCPCS_CD',): 579350.8697883942, ('clm_dates', 'provider'): 671621.2850168342, ('clm_dates', 'DGNS_CD'): 655866.5699742581, ('clm_dates', 'PRDCR_CD'): 573426.2830395221, ('clm_dates', 'HCPCS_CD'): 577424.5378240343, ('provider', 'DGNS_CD'): 667755.5469570069, ('provider', 'PRDCR_CD'): 599120.2055218299, ('provider', 'HCPCS_CD'): 595918.7768377154, ('DGNS_CD', 'PRDCR_CD'): 618940.1453773105, ('DGNS_CD', 'HCPCS_CD'): 613843.2555057921, ('PRDCR_CD', 'HCPCS_CD'): 578562.9685179077, ('clm_dates', 'provider', 'DGNS_CD'): 660097.6955758034, ('clm_dates', 'provider', 'PRDCR_CD'): 598602.4518220932, ('clm_dates', 'provider', 'HCPCS_CD'): 593549.6435593399, ('clm_dates', 'DGNS_CD', 'PRDCR_CD'): 612536.4561931454, ('clm_dates', 'DGNS_CD', 'HCPCS_CD'): 602094.9178627948, ('clm_dates', 'PRDCR_CD', 'HCPCS_CD'): 574173.9921150157, ('provider', 'DGNS_CD', 'PRDCR_CD')